In [ ]:
from google.colab import drive
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers[torch] -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
pip install accelerate -U


In [ ]:
# Set display options to show full content and set column width to max
pd.set_option('display.max_colwidth', 1000000000)
pd.set_option('display.max_columns', 500)

# Data Extraction

In [ ]:
import json  # Add this line to import the json module
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re


In [ ]:
# Define functions to process HTML text and JSON file
def process_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text().replace('\n', '').replace('\t', '')
    return cleaned_text



def process_json(file):
    # only the selected columns will remain after processing
    res_cols = [
        '_id', 'feed', 'title', 'url', 'pubdate', 'content',
        'tags', 'signature', 'created', 'updated', 'links',
        'authors', 'feedburner_origlink', 'summary',
        'guidislink', 'loki']

    keys_to_remove = [key for key in file if key not in res_cols]
    # delete columns intending to drop
    for key in keys_to_remove:
        del file[key]

    # Process each key if it exists
    if '_id' in file:
        file['_id'] = file['_id']['$oid']
    if 'feed' in file:
        file['feed'] = file['feed']['$oid']
    if 'pubdate' in file:
        file['pubdate'] = np.datetime64(file['pubdate']['$date'], 'ms')
    if 'content' in file:
        file['content'] = process_html(file['content'])
    if 'summary' in file:
        file['summary'] = process_html(file['summary'])
    if 'created' in file:
        file['created'] = np.datetime64(file['created']['$date'], 'ms')
    if 'updated' in file:
        file['updated'] = np.datetime64(file['updated']['$date'], 'ms')
    if 'authors' in file:
        file['authors'] = [list(x.values()) for x in file['authors']]

    return file;

In [ ]:
# Process the top 10 JSON files
processed_data = []
for file_path in all_files:
    # Open the JSON file and load its contents
    with open(file_path, "r") as file:
        data = json.load(file)

    # Process the JSON data
    processed_data.append(process_json(data))



<ipython-input-11-e8cb90e790b1>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
# Create a DataFrame from the processed data
df = pd.DataFrame(processed_data)

In [ ]:
df

,_id,feed,title,url,pubdate,content,tags,signature,created,updated,links,feedburner_origlink,summary,guidislink,authors
0,58b83d53df23b703bd69175d,56d609d1c180811310e52be2,"Richard Branson Announces New Space Company, T...",http://feeds.gawker.com/~r/gizmodo/full/~3/rwM...,2017-03-02 15:30:00,"Richard Branson Announces New Space Company, T...","[richard branson, virgin galactic, reverse mus...",a78907f66f72677273a5300b1898a65e91c48a7aee03e1...,2017-03-02 15:42:11.754,2017-03-02 15:42:11.833,[{'href': 'http://feeds.gawker.com/~r/gizmodo/...,http://gizmodo.com/richard-branson-announces-n...,“I hope in five years a reusable rocket will h...,False,[[Matt Novak]]
1,58ae1ff0df23b703bd552d4d,56d609d1c180811310e52bed,"No, Poppy Seeds Probably Aren't Going to Make ...",http://feeds.gawker.com/~r/lifehacker/vip/~3/p...,2017-02-22 23:01:59,"No, Poppy Seeds Probably Aren't Going to Make ...","[science, body science, food, poppy seeds, bag...",25a8ab87df630638641e8d3c546c1f6bffb3d8f2f57391...,2017-02-22 23:34:08.759,2017-02-22 23:34:08.835,[{'href': 'http://feeds.gawker.com/~r/lifehack...,http://lifehacker.com/no-poppy-seeds-probably-...,Tales of poppy seed-covered foods being the ca...,False,[[Patrick Allan]]
2,5984a8cadf23b703bdd21f15,56d609d1c180811310e52bef,Grab an Amazon Echo for half off today,https://www.engadget.com/2017/08/04/amazon-ech...,2017-08-04 16:39:00,Grab an Amazon Echo for half off todayEngadge...,"[alexa, amazon, amazonecho, gear, home]",34785804b4165b82d3654de3d26a0de3b457bb0848266a...,2017-08-04 17:03:06.612,2017-08-04 17:03:06.720,[{'href': 'https://www.engadget.com/2017/08/04...,NaN,"Amazon's been expanding its line of smart, Ale...",False,[[Swapna Krishna]]
3,5964c38fdf23b703bd9cff70,56d609d1c180811310e52bef,Vodafone shows again that own-brand phones can...,https://www.engadget.com/2017/07/11/vodafone-s...,2017-07-11 11:30:00,Vodafone shows again that own-brand phones ca...,"[gear, hands-on, mobile, smartn8, smartphone, ...",281cbcd3282e37979e205e6133f3e7e36e5af14afa696c...,2017-07-11 12:24:47.253,2017-07-11 12:24:47.376,[{'href': 'https://www.engadget.com/2017/07/11...,NaN,Vodafone's own-brand devices have been hit and...,False,[[Jamie Rigg]]
4,597b6adedf23b703bdc26cad,56d609d1c180811310e52beb,Apple’s Lisa Jackson to talk about the environ...,http://feedproxy.google.com/~r/Techcrunch/~3/1...,2017-07-28 15:47:41,Apple’s Lisa Jackson to talk about the environ...,"[TC, Lisa Jackson, Apple, Disrupt, disrupt sf,...",7d75baaef8afc9fa19a3d5fa865e1a6f8167a9a81f01aa...,2017-07-28 16:48:30.118,2017-07-28 16:48:30.202,[{'href': 'http://feedproxy.google.com/~r/Tech...,https://techcrunch.com/2017/07/28/apples-lisa-...,The tech industry has become so big that it n...,False,[[Romain Dillet]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13858,56f53855c18081352e6541c8,56d609d1c180811310e52beb,Encryption pioneer Martin Hellman talks securi...,http://feedproxy.google.com/~r/Techcrunch/~3/6...,2016-03-24 21:08:41,Turing Award Winner Martin Hellman On Cryptogr...,"[TC, martin hellman, Stanford University, Appl...",c55bbda837d7ded15a132d5e65cca0ec118f9a77975616...,2016-03-25 13:08:37.025,2016-03-25 13:08:37.103,[{'href': 'http://feedproxy.google.com/~r/Tech...,http://techcrunch.com/2016/03/24/turing-award-...,"Martin Hellman, Stanford Professor Emeritus o...",False,[[Nitish Kulkarni]]
13859,57259a15c1808103a0e73fd2,56d609d1c180811310e52bea,Obama's 'Red Wedding' joke proves he deserved ...,http://feeds.mashable.com/~r/Mashable/~3/xaix9...,2016-05-01 05:19:20,Obama's 'Red Wedding' joke proves he deserved ...,"[Game Of Thrones, Barack Obama, White House Co...",a2d375328ef75050f7236de8f5aa8e3637c76dad2fb26b...,2016-05-01 05:54:29.057,2016-05-01 05:54:29.139,[{'href': 'http://feeds.mashable.com/~r/Mashab...,http://mashable.com/2016/05/01/obama-red-weddi...,Tired of Senate Republicans refusing to approv...,False,[[Brittany Levine Beckman]]
13860,5729f357c180817cd22d0702,56d609d1c180811310e52be8,Three ways to optimize Puppe

In [ ]:
# Define the filename for the CSV file
csv_filename = '/content/drive/Shared drives/Capstone/tech/tech_csv_data.csv'

# Export the DataFrame to a CSV file in the specified directory
df.to_csv(csv_filename, index=False)

# Print confirmation message
print(f"CSV file saved to: {csv_filename}")

CSV file saved to: /content/drive/Shared drives/Capstone/tech/tech_csv_data.csv


In [ ]:
# Define the path to the CSV file
csv_filename = '/content/drive/Shared drives/Capstone/tech_csv_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_filename)

In [ ]:
df

_id                      feed  \
0      58b83d53df23b703bd69175d  56d609d1c180811310e52be2   
1      58ae1ff0df23b703bd552d4d  56d609d1c180811310e52bed   
2      5984a8cadf23b703bdd21f15  56d609d1c180811310e52bef   
3      5964c38fdf23b703bd9cff70  56d609d1c180811310e52bef   
4      597b6adedf23b703bdc26cad  56d609d1c180811310e52beb   
...                         ...                       ...   
13858  56f53855c18081352e6541c8  56d609d1c180811310e52beb   
13859  57259a15c1808103a0e73fd2  56d609d1c180811310e52bea   
13860  5729f357c180817cd22d0702  56d609d1c180811310e52be8   
13861  572fbe17c180817cd235d89c  56d609d1c180811310e52bea   
13862  57236bc3c1808103a0e3f3d1  56d609d1c180811310e52bea   

                                                                                                      title  \
0                                Richard Branson Announces New Space Company, Tries Pulling a Reverse Musk™   
1                                        No, Poppy Seeds Probably Aren't Going to Make You Fail a Drug Test   
2                                                                    Grab an Amazon Echo for half off today   
3                                              Vodafone shows again that own-brand phones can be good value   
4      Apple’s Lisa Jackson to talk about the environmental and social role of tech companies at Disrupt SF   
...                                                                                                     ...   
13858       Encryption pioneer Martin Hellman talks security, Apple, the FBI and the future of cryptography   
13859                    Obama's 'Red Wedding' joke proves he deserved that early look at 'Game of Thrones'   
13860                                                                         Three ways to optimize Puppet   
13861                                 Creating graffiti in VR looks so real, you can almost smell the fumes   
13862                                                           Netflix orders 'Punisher' standalone series   

                                                                                                                              url  \
0        http://feeds.gawker.com/~r/gizmodo/full/~3/rwMhGDT3aa0/richard-branson-announces-new-space-company-tries-pull-1792890909   
1      http://feeds.gawker.com/~r/lifehacker/vip/~3/pzjgoXvIMuc/no-poppy-seeds-probably-arent-going-to-make-you-fail-a-1792650401   
2                                                                 https://www.engadget.com/2017/08/04/amazon-echo-half-off-today/   
3                                                                   https://www.engadget.com/2017/07/11/vodafone-smart-own-brand/   
4                                                                       http://feedproxy.google.com/~r/Techcrunch/~3/18dL7NDCjvA/   
...                                                                                                                           ...   
13858                                                                   http://feedproxy.google.com/~r/Techcrunch/~3/6IV8uHMdfBw/   
13859                                                                       http://feeds.mashable.com/~r/Mashable/~3/xaix9qC8_P8/   
13860                                 http://feedproxy.google.com/~r/oreilly/radar/atom/~3/Ln4VfRPC1JE/three-ways-optimize-puppet   
13861                                                                       http://feeds.mashable.com/~r/Mashable/~3/55gYIZ2iOnQ/   
13862                                                                       http://feeds.mashable.com/~r/Mashable/~3/A_bsmAhYWtI/   

                   pubdate  \
0      2017-03-02 15:30:00   
1      2017-02-22 23:01:59   
2      2017-08-04 16:39:00   
3      2017-07-11 11:30:00   
4      2017-07-28 15:47:41   
...                    ...   
13858  2016-03-24 21:08:41   
13859  2016-05-01 05:19:20   
13860                  NaN   
13861  2016-05-08 21:07:15   
13862  2016-04-29 13:57:51   

    

In [ ]:
# Specify the path to the uploaded CSV file
csv_file_path = '/tech_csv_data.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [ ]:
df

_id                      feed  \
0      58b83d53df23b703bd69175d  56d609d1c180811310e52be2   
1      58ae1ff0df23b703bd552d4d  56d609d1c180811310e52bed   
2      5984a8cadf23b703bdd21f15  56d609d1c180811310e52bef   
3      5964c38fdf23b703bd9cff70  56d609d1c180811310e52bef   
4      597b6adedf23b703bdc26cad  56d609d1c180811310e52beb   
...                         ...                       ...   
13858  56f53855c18081352e6541c8  56d609d1c180811310e52beb   
13859  57259a15c1808103a0e73fd2  56d609d1c180811310e52bea   
13860  5729f357c180817cd22d0702  56d609d1c180811310e52be8   
13861  572fbe17c180817cd235d89c  56d609d1c180811310e52bea   
13862  57236bc3c1808103a0e3f3d1  56d609d1c180811310e52bea   

                                                                                                      title  \
0                                Richard Branson Announces New Space Company, Tries Pulling a Reverse Musk™   
1                                        No, Poppy Seeds Probably Aren't Going to Make You Fail a Drug Test   
2                                                                    Grab an Amazon Echo for half off today   
3                                              Vodafone shows again that own-brand phones can be good value   
4      Apple’s Lisa Jackson to talk about the environmental and social role of tech companies at Disrupt SF   
...                                                                                                     ...   
13858       Encryption pioneer Martin Hellman talks security, Apple, the FBI and the future of cryptography   
13859                    Obama's 'Red Wedding' joke proves he deserved that early look at 'Game of Thrones'   
13860                                                                         Three ways to optimize Puppet   
13861                                 Creating graffiti in VR looks so real, you can almost smell the fumes   
13862                                                           Netflix orders 'Punisher' standalone series   

                                                                                                                              url  \
0        http://feeds.gawker.com/~r/gizmodo/full/~3/rwMhGDT3aa0/richard-branson-announces-new-space-company-tries-pull-1792890909   
1      http://feeds.gawker.com/~r/lifehacker/vip/~3/pzjgoXvIMuc/no-poppy-seeds-probably-arent-going-to-make-you-fail-a-1792650401   
2                                                                 https://www.engadget.com/2017/08/04/amazon-echo-half-off-today/   
3                                                                   https://www.engadget.com/2017/07/11/vodafone-smart-own-brand/   
4                                                                       http://feedproxy.google.com/~r/Techcrunch/~3/18dL7NDCjvA/   
...                                                                                                                           ...   
13858                                                                   http://feedproxy.google.com/~r/Techcrunch/~3/6IV8uHMdfBw/   
13859                                                                       http://feeds.mashable.com/~r/Mashable/~3/xaix9qC8_P8/   
13860                                 http://feedproxy.google.com/~r/oreilly/radar/atom/~3/Ln4VfRPC1JE/three-ways-optimize-puppet   
13861                                                                       http://feeds.mashable.com/~r/Mashable/~3/55gYIZ2iOnQ/   
13862                                                                       http://feeds.mashable.com/~r/Mashable/~3/A_bsmAhYWtI/   

                   pubdate  \
0      2017-03-02 15:30:00   
1      2017-02-22 23:01:59   
2      2017-08-04 16:39:00   
3      2017-07-11 11:30:00   
4      2017-07-28 15:47:41   
...                    ...   
13858  2016-03-24 21:08:41   
13859  2016-05-01 05:19:20   
13860                  NaN   
13861  2016-05-08 21:07:15   
13862  2016-04-29 13:57:51   

    

# Cleaning the dirty text!

In [ ]:
df['content'] = df['content'].astype(str)

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove sentences containing "Click Here to find out more"
    sentences = text.split('.')
    cleaned_sentences = [sentence.strip() for sentence in sentences if 'Click Here to find out more' not in sentence]
    cleaned_text = '. '.join(cleaned_sentences)

    # Remove words with more than 20 characters
    text = ' '.join(word for word in text.split() if len(word) <= 20)

    return text


# Apply the cleaning function to the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)


In [ ]:
def remove_click_here_sentence(text):
    # Define a regular expression pattern to match sentences
    sentence_pattern = r'([^.!?]+[.!?])'

    # Find all sentences in the text
    sentences = re.findall(sentence_pattern, text)

     # Define phrases to be removed, ignoring case
    phrases_to_remove = ['click here to find out', 'all rights reserved', 'using cookies', 'menuopen', 'powered by its own proprietary technology, mashable is the go-to source for tech, digital culture and', 'reproduction without explicit permission is prohibited',
                         'addresssubscribe', '©', ' gizmodo media groupkinja is in read-only mode', 'we are working to restore service', 'need help?', 'send us a tip!', 'policyterms of servicepowered by wordpress']

    # Remove sentences containing the phrases
    filtered_sentences = [sentence.strip() for sentence in sentences
                          if all(phrase.lower() not in sentence.lower() for phrase in phrases_to_remove)]


    # Join the remaining sentences
    cleaned_text = ' '.join(filtered_sentences)

    return cleaned_text

df['cleaned_content'] = df['cleaned_content'].apply(remove_click_here_sentence)


In [ ]:
df[['content', 'cleaned_content']].head(50)

content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# Testing with Hugging Face Transformer Library- summarization Pipeline (by default BART)
-- works well!


In [ ]:
!pip install transformers

In [ ]:
#import dependencies
from transformers import pipeline

In [ ]:
#Load summarization pipeline
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
article = df['cleaned_content'].tail(1).tolist()
article

["Jon Bernthal as Frank Castle, A. K. A. the Punisher, in 'Daredevil' Season 2Image: netflixBy Proma Khosla2016-04-29 13:57:51 UTCHell's Kitchen continues to burn. With success of Daredevil, the critical acclaim of Jessica Jones, and mounting excitement for Luke Cage and Iron Fist — all leading up to a Defenders miniseries — Marvel is adding another name to its slate of Netflix properties. SEE ALSO: 'Daredevil' Season 2 is crowded, which just means there's more to loveThe new series will focus on The Punisher, and will star Jon Bernthal in the title role — reprising a part he first played on Daredevil Season 2. Hannibal's Steve Lightfoot will serve as showrunner. News of the show was first reported by Entertainment Weekly and confirmed by Netflix on Friday. The Punisher was first introduced in Frank Miller's Daredevil comics as a vengeful and violent vigilante – a darker shade of Daredevil and a warning of what Matt Murdock could become if he veered off his chosen path. Castle is disil

In [ ]:
summarizer(article, max_length=150, min_length=50, do_sample=False)


[{'summary_text': " New series will star Jon Bernthal in the title role of The Punisher . Hannibal's Steve Lightfoot will serve as showrunner . The series doesn't yet have a premiere date . It will star Bernthal as the Punisher in the role he first played on Daredevil Season 2 ."}]

# Train and Fine-tune BART

In [ ]:
from transformers import BartTokenizer
import torch


In [ ]:
#preprocessing first

In [ ]:
# Load BART tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
df

NameError: name 'df' is not defined

In [ ]:
# Convert Series of tensors into a list of dictionaries
tokenized_texts = [tokenizer(x, truncation=True, padding='max_length', max_length=512, return_tensors="pt") for x in df['cleaned_content']]

# Get maximum sequence length
max_length = max(len(tokenized_texts[i]['input_ids'][0]) for i in range(len(tokenized_texts)))


# Pad sequences to maximum length
input_ids = torch.stack([torch.cat([tokenized_texts[i]['input_ids'][0], torch.zeros(max_length - len(tokenized_texts[i]['input_ids'][0]), dtype=torch.long)]) for i in range(len(tokenized_texts))])
attention_masks = torch.stack([torch.cat([tokenized_texts[i]['attention_mask'][0], torch.zeros(max_length - len(tokenized_texts[i]['attention_mask'][0]), dtype=torch.long)]) for i in range(len(tokenized_texts))])


In [ ]:
#fine-tuning the pre-trained BART model on my data.

In [ ]:
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler


In [ ]:
# Load pre-trained BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


In [ ]:
# Define training parameters
batch_size = 1
num_epochs = 3
learning_rate = 2e-5

# Convert input tensors to PyTorch Dataset
dataset = TensorDataset(input_ids, attention_masks)

# Create DataLoader for batching and shuffling data
dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size)

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)


In [ ]:
# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}

        # Forward pass
        outputs = model(**inputs)

        # No loss calculation in self-supervised learning


    print(f"Epoch {epoch+1}/{num_epochs} completed")

In [ ]:
# Save both the model and tokenizer
model.save_pretrained('/content/drive/Shared drives/Capstone/fine_tuned_bart_model')
tokenizer.save_pretrained('/content/drive/Shared drives/Capstone/fine_tuned_bart_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


KeyboardInterrupt: 

In [ ]:
# Specify the directory path to save the model and tokenizer
directory_path = '/tech_models2/'

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Set the paths to save the model and tokenizer within the directory
model_path = os.path.join(directory_path, 'fine_tuned_bart_model')
tokenizer_path = os.path.join(directory_path, 'fine_tuned_bart_tokenizer')

# Save the model and tokenizer within the directory
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/tech_models2/fine_tuned_bart_tokenizer/tokenizer_config.json',
 '/tech_models2/fine_tuned_bart_tokenizer/special_tokens_map.json',
 '/tech_models2/fine_tuned_bart_tokenizer/vocab.json',
 '/tech_models2/fine_tuned_bart_tokenizer/merges.txt',
 '/tech_models2/fine_tuned_bart_tokenizer/added_tokens.json')

# Testing the PreTrained BART Model

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer


In [ ]:

# Load the fine-tuned BART model
model = BartForConditionalGeneration.from_pretrained(model_path)

# Load the fine-tuned BART tokenizer
tokenizer = BartTokenizer.from_pretrained(tokenizer_path)

In [ ]:
input_text = df['cleaned_content'].head(1).iloc[-1]
input_text

'Richard Branson Announces New Space Company, Tries Pulling a Reverse GuideRichard Branson Announces New Space Company, Tries Pulling a Reverse Musk™Matt Novak11 minutes agoFiled to: richard bransonvirgin galacticreverse muskvirgin to KinjaToggle Conversation toolsGo to permalinkRichard Branson at a mental health conference in April 2016 (Photo by Alex Wong/Getty Images)“I hope in five years a reusable rocket will have been developed which can take up to 10 people at a time to stay at the Virgin Hotel for two weeks,” Richard Branson proudly announced back in still waiting on Branson’s Virgin Galactic to take tourists into space, despite the promises of the past two decades that he would do just that. But Branson has a new idea for the meantime: Why not launch small satellites into space? Called Virgin Orbit, Branson’s newest space venture will be led by former Boeing executive Dan Hart. “By offering low cost and frequent service to space, the team is revolutionizing the small satellite

In [ ]:

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)["input_ids"]

# Generate the summary
summary_ids = model.generate(input_ids, num_beams=4, min_length=30, max_length=150, early_stopping=True)

# Decode the summary tokens back into text
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print or use the generated summary
print("Generated Summary:", summary_text)

Generated Summary: Richard Branson Announces New Space Company, Tries Pulling a Reverse Guide. Elon Musk founded SpaceX in 2002 with bold dreams of eventually propelling humans into space. Musk realized that you had to start with the more modest stuff.


# Pre-Processing & Modeling using BART

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
# Load the pre-trained BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [ ]:
# Define a Dataset class to load and preprocess the data
class NewsDataset(Dataset):
    def __init__(self, df, max_input_length=1024, max_output_length=150):
        self.df = df
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_text = self.df.iloc[idx]['content']
        target_text = self.df.iloc[idx]['summary']

        # Tokenize the input and target text
        input_tokens = tokenizer(input_text, max_length=self.max_input_length, padding='max_length', truncation=True, return_tensors='pt')
        target_tokens = tokenizer(target_text, max_length=self.max_output_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].flatten(),
            'attention_mask': input_tokens['attention_mask'].flatten(),
            'labels': target_tokens['input_ids'].flatten(),
            'decoder_attention_mask': target_tokens['attention_mask'].flatten()
        }


In [ ]:
df = df[['content', 'summary']]

In [ ]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define batch size and number of epochs
batch_size = 2
num_epochs = 3

# Create Dataset and DataLoader for training and validation sets
train_dataset = NewsDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = NewsDataset(val_df)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Load the pre-trained BART model for conditional generation
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Fine-tune the pre-trained model on the training data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc='Epoch %d/%d' % (epoch+1, num_epochs)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        decoder_attention_mask = batch['decoder_attention_mask'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validate the model
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            decoder_attention_mask = batch['decoder_attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
            loss = outputs.loss

            val_loss += loss.item()

        val_loss /= len(val_loader)

    print('Epoch %d/%d - Train Loss: %.4f - Validation Loss: %.4f' % (epoch+1, num_epochs, train_loss, val_loss))

# Save the trained model
model.save_pretrained('bart_news_summarizer')

Epoch 1/3:  75%|███████▌  | 3/4 [03:43<01:13, 73.02s/it]

# Using BERT!

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch

In [ ]:

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for masked language modeling
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

In [ ]:
df = df.loc[:, ['cleaned_content']]
df

,cleaned_content
0,"Richard Branson Announces New Space Company, T..."
1,"No, Poppy Seeds Probably Aren't Going to Make ..."
2,Grab an Amazon Echo for half off todayEngadget...
3,Vodafone shows again that own-brand phones can...
4,Apple’s Lisa Jackson to talk about the environ...
...,...
13858,Turing Award Winner Martin Hellman On Cryptogr...
13859,President Barack Obama throws in a 'Game of Th...
13860,"By Jo Rhett April 25, 2016Departure board at n..."
13861,By Adam Rosenberg2016-05-08 21:07:15 UTCIf vir...


In [ ]:
articles = df['content'].tolist()

In [ ]:
#test with top 10
articles = df['cleaned_content'].tail(100).tolist()
#articles = df['content'].apply(str).tolist()


In [ ]:
# Tokenize the articles and convert them to tensors
inputs = tokenizer(articles, return_tensors="pt", padding=True, truncation=True)


In [ ]:
# Fine-tune BERT on your dataset
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Define optimizer
epochs = 3  # Define number of training epochs
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs["input_ids"])  # Masked language modeling task
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


In [ ]:

# Save both the model and tokenizer
model.save_pretrained('/content/drive/Shared drives/Capstone/fine_tuned_bert_news_summarization')
tokenizer.save_pretrained('/content/drive/Shared drives/Capstone/fine_tuned_bert_news_summarization')



# Testing the fine-tuned BERT Model

In [ ]:
# Load the tokenizer and model
tokenizer_path = '/content/drive/Shared drives/Capstone/fine_tuned_bert_news_summarization'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForMaskedLM.from_pretrained(tokenizer_path)


In [ ]:
# Set the model in evaluation mode
model.eval()

# Example news article text
def escape_single_quotes(text):
    return text.replace("'", "\\'")

news_article = escape_single_quotes("""US self-driving car bill heads to the House floorEngadget Login Home Gear Gaming Entertainment Tomorrow Video Reviews Find a Product Buyers Guide Log in Sign up Show More ResultsEngadget3 related articles Login Motorcycle helmets finally get decent heads-up display navigationFord’s Chariot rideshare service expands to New York CityTesla will livestream the first Model 3 deliveries tomorrow Latest in Gaming US self-driving car bill heads to the House floor 8m agoView 'Wolfenstein II' DLC adds three new Nazi-punching characters 2h agoView The top Twitch clip involves a horror game and Jack Daniels 6h agoView Blizzard is increasing ‘Overwatch’ penalties for bad behavior 7h agoViewImage credit: Nathan Ingraham/Engadgetsave Saveshare US self-driving car bill heads to the House floorIt's another tiny step forward for autonomous vehicles. Rob LeFebvre, @roblef 8m ago in TransportationComments 5SharesShareTweetShareSaveNathan Ingraham/EngadgetLast month, a Senate committee created a proposal to allow autonomous vehicles onto the roads under specific safety and "tech neutral" requirements. Now that the bill has hit House of Representatives, the bipartisan Energy and Commerce Committee voted to send it along to the full chamber. The Safely Ensuring Lives Future Deployment and Research In Vehicle Evolution Act, or SELF DRIVE, is aimed at allowing companies like Uber and Google to test up to 100,000 autonomous vehicles across the country. While we're far from an actual bill, this seems like good forward movement.If driverless cars are ever going to gain a foothold on the road, they'll need to be as safe as (or safer that) current automobiles are. Companies like Lyft, GM and Toyota have been lobbying for this type of legislation for a while now. The Department of Transportation will need to ensure this if it wants to exempt the vehicles from federal rules that govern current cars, like the requirement they have steering wheels, for example. The proposed bill, which isn't yet scheduled for full House consideration, also bars states from setting their own possibly conflicting regulations around the design, operation and software components of self-driving cars. States will be able to restrict autonomous vehicles on public roads in some way, according to ReCode. Via: ReCode Source: US House of RepresentativesIn this article: autonomous, AutonomousVehicles, gaming, google, legistlation, politics, SelfDrivingCar, transportation, uber, waymo 5SharesShareTweetShareSaveCommentsSign In 8m 8m ago in Transportation US self-driving car bill heads to the House floor It's another tiny step forward for autonomous vehicles. By R. LeFebvre,  8m ago save Save share View 22m 22m ago in Internet Flash was useful, but developers are glad it’s on the death march And they say it's still pervasive enough to warrant an extended phase out. By M. Locklear,  22m ago save Save share View 39m 39m ago in Internet Apple granted approval to test its 5G wireless technology It’s focusing on millimeter-wave broadband. By M. Locklear,  39m ago save Save share View 52m 52m ago in Transportation Motorcycle helmets finally get decent heads-up display navigation A Nuviz is handy a companion for weekend rides. By R. Baldwin,  52m ago save Save share View 1h 1h ago in Personal Computing Scientists can turn crystals into reusable electrical conductors They last for up to a year with no degradation. By S. Krishna,  1h ago save Save share View More Stories From around the web About About EngadgetAbout Our AdsAdvertiseBrand KitContact UsRSS Feed Sections ReviewsGearGamingEntertainmentTomorrowVideoPodcastsDealsFind a Product Contribute Comment GuidelinesSend us a tip!Support International Español繁體中文简体中文日本版DeutschlandUK Apps AndroidiPhoneiPadJoin Our NewsletterSubscribeFollow Us© 2017 Oath Inc. All rights reserved.Reprints and PermissionsPrivacy PolicyTerms of UseTrademarksAdvertiseear iconeye icontext file""")
news_article

'US self-driving car bill heads to the House floorEngadget Login Home Gear Gaming Entertainment Tomorrow Video Reviews Find a Product Buyers Guide Log in Sign up Show More ResultsEngadget3 related articles Login Motorcycle helmets finally get decent heads-up display navigationFord’s Chariot rideshare service expands to New York CityTesla will livestream the first Model 3 deliveries tomorrow Latest in Gaming US self-driving car bill heads to the House floor 8m agoView \\\'Wolfenstein II\\\' DLC adds three new Nazi-punching characters 2h agoView The top Twitch clip involves a horror game and Jack Daniels 6h agoView Blizzard is increasing ‘Overwatch’ penalties for bad behavior 7h agoViewImage credit: Nathan Ingraham/Engadgetsave Saveshare US self-driving car bill heads to the House floorIt\\\'s another tiny step forward for autonomous vehicles. Rob LeFebvre, @roblef 8m ago in TransportationComments 5SharesShareTweetShareSaveNathan Ingraham/EngadgetLast month, a Senate committee created a 

In [ ]:
# Tokenize the input text
inputs = tokenizer.encode_plus(news_article, return_tensors="pt", max_length=512, truncation=True)

# Generate predictions
with torch.no_grad():
    outputs = model(**inputs)

# Decode the output tokens to get the summary
predicted_tokens = outputs.logits.argmax(dim=-1)[0]
predicted_summary = tokenizer.decode(predicted_tokens, skip_special_tokens=True)

# Print the summary
print("Generated Summary:", predicted_summary)

Generated Summary: the us self - driving car bill heads to the house floorengadget login home gear gaming entertainment tomorrow video reviews find a product buyers guide log in sign up show more resultsengadget3 related articles login motorcycle helmets finally get decent heads - up display navigationford's chariot rideshare service expands to new york citytesla will livestream the first model 3 deliveries tomorrow latest in gaming us self - driving car bill heads to the house floor 8m agoview \'wolfenstein ii \'dlc adds three new nazi - punching characters 2h agoview the top twitch clip involves a horror game and jack daniels 6h agoview blizzard is increasing and overwatch increased penalties for bad behavior 7h agoviewimage credit : nathan ingraham / engadgetsave saveshare us self - driving car bill heads to the house floorit \'s another tiny step forward for autonomous vehicles. rob lefebvre, @ roblef 8m ago in transportationcomments 5sharessharetweetsharesavenathan ingraham / enga

In [ ]:
# Tokenize the input text with shorter maximum length
inputs = tokenizer.encode_plus(news_article, return_tensors="pt", max_length=100, truncation=True)

# Generate predictions
with torch.no_grad():
    outputs = model(**inputs)

# Decode the output tokens to get the summary
predicted_tokens = outputs.logits.argmax(dim=-1)[0]
predicted_summary = tokenizer.decode(predicted_tokens, skip_special_tokens=True)

# Print the summary
print("Generated Summary:", predicted_summary)

Generated Summary: ##s us self - driving car bill heads to the house floorengadget login home gear gaming entertainment tomorrow video reviews find a product buyers guide log in sign up show more resultsengadget3 related articles login motorcycle helmets finally get decent heads - up display navigationford's chariot rideshare service expands to new york citytesla will livestream the first model 3 deliveries tomorrow latest in gaming us self - driving car bill heads to the house floor 8m agoview \ '!.
